# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [102]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [103]:
# Hacé la separación en esta celda
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [104]:
# Creá en esta celda la variable param_grid
param_grid = [{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}]

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [105]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [106]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [107]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [108]:
# Mostrá los grid_scores en esta celda
np.sqrt(-grid_search.cv_results_['mean_train_score'])

array([31262.7320801 , 31223.76019147, 30196.5398551 , 30464.81785195,
       29832.38605568, 30457.49703695, 31116.76383437, 29186.58265136,
       28982.48454558, 29649.52102899, 30374.33912251, 30193.36170339,
       29364.8011195 , 29035.83179866, 28006.77742724, 30106.94383724,
       30097.35415678, 27069.87267999, 28574.49077856, 28434.77622335,
       30494.87226804, 28727.26091043, 27253.28999779, 27096.43550584,
       27050.67236841])

In [109]:
np.sqrt(-grid_search.cv_results_['mean_test_score'])

array([31309.57850585, 31267.12160334, 30488.28562686, 30390.25891769,
       29866.71009919, 30322.74061448, 31142.69024368, 29176.07891481,
       29103.59421489, 29590.6537991 , 30390.70967028, 30320.98111145,
       29345.53392924, 29300.19341846, 28259.80189735, 30428.14465867,
       30100.38240479, 27403.11424655, 28819.34651145, 28660.25062404,
       31024.85349496, 29334.83427828, 27859.87131326, 27397.71986906,
       27580.63847613])

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [110]:
# Mostrás los resultados en esta celda
grid_search.best_params_

{'max_depth': 5, 'max_features': 4}

Convertimos a RMSE.

In [111]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [112]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [113]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

24307.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [114]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [115]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [116]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,102915.039,22915.039
1,128000.000,125989.041,2010.959
2,150000.000,138390.657,11609.343
3,85000.000,109839.825,24839.825
4,135000.000,138390.657,3390.657
5,135000.000,115001.396,19998.604
6,68000.000,127569.841,59569.841
7,110000.000,148829.177,38829.177
8,134000.000,127569.841,6430.159
9,110000.000,91620.291,18379.709
